# VerifAI : Comparaison des Modèles RandomForest et XGBoost sur le Dataset EMBER

Ce notebook est dédié au projet **VerifAI**, une solution de détection statique de virus basée sur le machine learning. 

## Objectifs du Notebook
1. **Entraînement de Modèles** :
   - Entraîner deux modèles de machine learning : **RandomForest** et **XGBoost**.
   - Utiliser le **dataset EMBER**, un benchmark reconnu pour l'analyse statique des fichiers exécutables.
   
2. **Comparaison des Performances** :
   - Évaluer les performances des deux modèles à l'aide de métriques clés : précision, rappel, score F1, et courbe ROC.
   - Identifier le modèle le plus performant pour l'intégration dans notre application **VerifAI**.

3. **Exportation des Modèles** :
   - Sauvegarder les modèles entraînés pour une utilisation directe dans notre pipeline de détection via Flask.

## Contexte
**VerifAI** repose sur une approche de détection proactive des malwares en utilisant des techniques de machine learning. Ce notebook constitue une étape clé pour sélectionner le meilleur modèle capable de détecter efficacement les fichiers malveillants, tout en minimisant les faux positifs. Les modèles exportés à partir de ce notebook seront directement utilisés dans l'application web.

> **Note** : Le dataset **EMBER** utilisé ici comprend des caractéristiques statiques extraites des exécutables, ce qui en fait une solution robuste pour la détection de virus sans exécution dynamique des fichiers.

--- 

Suivez les étapes de ce notebook pour découvrir les performances des modèles et préparer les fichiers nécessaires pour l'application **VerifAI** !


## Analyse de dataset:

In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
# Définir le chemin des données
data_dir = "/kaggle/input/ember-for-static-malware-analysis/"  # Chemin vers le dossier contenant les fichiers .dat

# Examiner les dimensions globales des fichiers
def dataset_info(data_dir):
    """
    Affiche les dimensions des fichiers de données EMBER (X_train, y_train, etc.)

    Args:
        data_dir (str): Chemin vers le dossier contenant les fichiers .dat
    """
    # Charger X_train
    X_train_path = os.path.join(data_dir, "X_train.dat")
    y_train_path = os.path.join(data_dir, "y_train.dat")

    if os.path.exists(X_train_path) and os.path.exists(y_train_path):
        # Obtenir les dimensions globales (nombre de lignes, colonnes)
        X_train_size = os.path.getsize(X_train_path) / np.dtype(np.float32).itemsize
        y_train_size = os.path.getsize(y_train_path) / np.dtype(np.float32).itemsize

        columns = 2381  # Taille par ligne pour X_train (selon spécifications EMBER)
        total_rows_X_train = int(X_train_size / columns)
        total_rows_y_train = int(y_train_size)

        print(f"Dataset Information:")
        print(f"- X_train: {total_rows_X_train} rows, {columns} columns")
        print(f"- y_train: {total_rows_y_train} rows")
        print(f"Maximum train_size possible: {min(total_rows_X_train, total_rows_y_train)}")
    else:
        print("X_train.dat or y_train.dat not found in the specified directory.")

# Afficher les informations
dataset_info(data_dir)


## Lancement de l'entrainement random forest


In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Configurations des chemins
data_dir = "/kaggle/input/ember-for-static-malware-analysis/"  # Modifier selon l'emplacement des données

# Charger les données d'entraînement
train_size = 800000
columns = 2381

X_data = np.memmap(os.path.join(data_dir, "X_train.dat"), dtype=np.float32, mode="r", shape=(train_size, columns))
y_data = np.memmap(os.path.join(data_dir, "y_train.dat"), dtype=np.float32, mode="r", shape=(train_size,))

# Filtrer les échantillons valides (labels -1 sont ignorés)
valid_rows = y_data != -1
X_data = X_data[valid_rows]
y_data = y_data[valid_rows]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Initialiser le modèle Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

# Entraîner le modèle
print("Training Random Forest model...")
start_time = time.time()  # Début du chronomètre
rf_model.fit(X_train, y_train)
end_time = time.time()  # Fin du chronomètre
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Prédire sur les données de test
print("Predicting on test set...")
y_pred = rf_model.predict(X_test)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Afficher le rapport de classification
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["SAFE", "MALWARE"]))

# Calculer et afficher la matrice de confusion
print("Plotting confusion matrix...")
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["SAFE", "MALWARE"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matrice de confusion - Random Forest")
plt.show()


## Entrainement avec XGboost

In [ ]:
import os
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import time

# Configurations des chemins
data_dir = "/kaggle/input/ember-for-static-malware-analysis/"  # Modifier selon l'emplacement des données

# Charger les données d'entraînement
train_size = 800000
columns = 2381

X_data = np.memmap(os.path.join(data_dir, "X_train.dat"), dtype=np.float32, mode="r", shape=(train_size, columns))
y_data = np.memmap(os.path.join(data_dir, "y_train.dat"), dtype=np.float32, mode="r", shape=(train_size,))

# Filtrer les échantillons valides (labels -1 sont ignorés)
valid_rows = y_data != -1
X_data = X_data[valid_rows]
y_data = y_data[valid_rows]

# Diviser le dataset en 80% entraînement et 20% test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Initialiser le modèle XGBoost
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    tree_method="gpu_hist",  # Utiliser "gpu_hist" si un GPU est disponible
    random_state=42,
    n_jobs=-1
)

# Entraîner le modèle
print("Training XGBoost model...")
start_time = time.time()  # Début du chronométrage

xgb_model.fit(X_train, y_train)

end_time = time.time()  # Fin du chronométrage
training_time = end_time - start_time

print(f"Training completed in {training_time:.2f} seconds.")

# Prédire sur les données de test
print("Predicting on test set...")
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Afficher le rapport de classification
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["SAFE", "MALWARE"]))


In [ ]:
import seaborn as sns
# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["SAFE", "MALWARE"], yticklabels=["SAFE", "MALWARE"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
